In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from astropy.coordinates import SkyCoord, Galactic, Galactocentric
import astropy.units as u
import astropy.constants as const

Step 1: Read in Gaia/ASPCAP Crossmatch File and isolate specific columns

In [2]:
file_path = '/Users/giovannigollotti/Downloads/SDSS_Files/ASPCAP_Gaia_00311_Crossmatch.csv'

df = pd.read_csv(file_path)

/var/folders/7b/jsrl76yd6v1b3v92fpy66jb00000gn/T/ipykernel_17019/2727656815.py:3: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [3]:
list(df.columns)

['sdss_id',
 'sdss4_apogee_id',
 'gaia_dr2_source_id',
 'gaia_dr3_source_id',
 'tic_v8_id',
 'healpix',
 'lead',
 'version_id',
 'catalogid',
 'catalogid21',
 'catalogid25',
 'catalogid31',
 'n_associated',
 'n_neighborhood',
 'sdss5_target_flags',
 'sdss4_apogee_target1_flags',
 'sdss4_apogee_target2_flags',
 'sdss4_apogee2_target1_flags',
 'sdss4_apogee2_target2_flags',
 'sdss4_apogee2_target3_flags',
 'sdss4_apogee_member_flags',
 'sdss4_apogee_extra_target_flags',
 'ra_1',
 'dec_1',
 'l_1',
 'b_1',
 'plx',
 'e_plx',
 'pmra_1',
 'e_pmra',
 'pmde',
 'e_pmde',
 'gaia_v_rad',
 'gaia_e_v_rad',
 'g_mag',
 'bp_mag',
 'rp_mag',
 'j_mag',
 'e_j_mag',
 'h_mag',
 'e_h_mag',
 'k_mag',
 'e_k_mag',
 'ph_qual',
 'bl_flg',
 'cc_flg',
 'w1_mag',
 'e_w1_mag',
 'w1_flux',
 'w1_dflux',
 'w1_frac',
 'w2_mag',
 'e_w2_mag',
 'w2_flux',
 'w2_dflux',
 'w2_frac',
 'w1uflags',
 'w2uflags',
 'w1aflags',
 'w2aflags',
 'mag4_5',
 'd4_5m',
 'rms_f4_5',
 'sqf_4_5',
 'mf4_5',
 'csf',
 'zgr_teff',
 'zgr_e_teff',
 '

In [4]:
df_cross_gaia = df[['sdss_id', 'gaia_dr3_source_id', 'healpix', 'catalogid31', 'ra_1', 'dec_1', 'teff', 'logg', 'bp_rp', 'fe_h', 'n_h',
                    'o_h','parallax', 'l_2', 'b_2', 'pm', 'pmra_2', 'pmdec', 'dr2_radial_velocity']]

#l_2 = galactic longitude (determined with Gaia)
#b_2 = galactic latitude (determined with Gaia)

In [5]:
#Deleting rows with np.nan values
df_cross_gaia = df_cross_gaia.dropna()
df_cross_gaia = df_cross_gaia.reset_index(drop=True) #Resetting index values

#Deleting rows where the Gaia DR3 ID == -1
gaia_id = np.array(df_cross_gaia['gaia_dr3_source_id'])
bad_gaia = gaia_id == -1
df_cross_gaia = df_cross_gaia[~bad_gaia]

df_cross_gaia = df_cross_gaia.reset_index(drop=True)

In [6]:
df_cross_gaia[:15] #All the mass values equate to np.nan, which is why we can't include the mass column

,sdss_id,gaia_dr3_source_id,healpix,catalogid31,ra_1,dec_1,teff,logg,bp_rp,fe_h,n_h,o_h,parallax,l_2,b_2,pm,pmra_2,pmdec,dr2_radial_velocity
0,114877950,18305872170309888,88375,63050396551579641,38.796894,5.596680,4919.6875,3.649784,1.113155,-0.428451,-0.486,-0.079161,1.641960,164.266403,-48.768841,46.431534,1.628700,-46.402959,5.771782
1,114877946,18305184975542656,88375,63050396551579627,38.847992,5.589650,6056.3066,4.438146,0.768336,-0.171147,-0.672,-0.280236,3.572811,164.334934,-48.744455,35.129265,-18.284791,-29.995527,11.636775
2,114877964,18308350366843648,88375,63050396551579682,38.863964,5.692591,6014.9100,3.887858,0.748663,-0.096147,-0.044,-0.135236,2.964072,164.262177,-48.652025,30.344755,-7.370606,-29.436004,-9.570927
3,114877081,18102531238471680,88887,63050396551575847,38.879242,5.271588,4757.8190,2.997648,1.243318,0.210549,0.448,0.148839,1.840202,164.659783,-48.982072,10.690163,-1.825898,-10.533075,-11.611487
4,114877962,18307800611028096,88375,63050396551579676,38.955757,5.695669,5620.2060,3.439589,0.780944,-0.104451,0.062,-0.298161,2.230055,164.370863,-48.595533,34.986893,33.420202,-10.352424,21.804190
5,114878130,18338827454939392,87863,63050396551580265,38.960840,5.885618,4767.5757,2.649483,1.240575,-0.020451,0.380,-0.058161,1.073568,164.207523,-48.439328,17.215813,16.318681,-5.484964,28.693666
6,114877061,18099546236637696,88887,63050396551575782,39.061424,5.394395,4856.0530,3.386672,1.201393,0.017549,0.130,0.000839,1.656024,164.770128,-48.775387,21.874280,-11.647756,-18.515236,-24.650982
7,114877011,18087623407428096,88887,63050396551575585,39.153690,5.216049,4893.7130,3.593636,1.119321,-1.430451,-1.247,-1.391161,0.791841,165.043903,-48.863439,24.979483,16.725639,-18.553369,-135.893340
8,114877074,18101161144336128,88887,63050396551575823,39.257620,5.471385,4857.5880,2.464972,1.201082,-0.318451,-0.161,-0.352161,0.960785,164.937748,-48.596978,11.727335,-9.013310,-7.502708,-45.550793
9,114877282,18144828076524800,88376,63050396551576623,39.266880,5.746507,4887.8490,2.473463,1.177738,-0.351451,-0.118,-0.255161,0.762328,164.701071,-48.370776,2.415549,1.756939,1.657722,2.840596


Step 2: Isolate the nitrogen-rich stars with reliable APOGEE abundances (AKA teff > 4500)

In [7]:
#Isolating stars with reliable temperatures
teff = np.array(df_cross_gaia['teff'])
teff_mask = teff > 4500

df_cross_gaia = df_cross_gaia[teff_mask]
df_cross_gaia = df_cross_gaia.reset_index(drop=True)

In [8]:
#Calculating nitrogen abundances
n_abundances = []

for i in range(len(df_cross_gaia['n_h'])):
    n_fe_val = df_cross_gaia['n_h'][i] - df_cross_gaia['fe_h'][i]
    n_abundances.append(n_fe_val)

n_fe_vals = np.array(n_abundances)

In [9]:
#Isolating nitrogen-rich stars
n_abund_mask = n_fe_vals >= 0.5

df_cross_gaia = df_cross_gaia[n_abund_mask]
df_cross_gaia = df_cross_gaia.reset_index(drop=True)

In [10]:
df_cross_gaia[:15]

,sdss_id,gaia_dr3_source_id,healpix,catalogid31,ra_1,dec_1,teff,logg,bp_rp,fe_h,n_h,o_h,parallax,l_2,b_2,pm,pmra_2,pmdec,dr2_radial_velocity
0,114876867,18047864894859136,89400,63050396551574904,39.364098,5.292768,5965.7430,3.904995,0.695686,-0.363451,0.398,-0.218161,1.514129,165.228239,-48.676291,10.255045,-8.348716,-5.955241,-21.060333
1,114876879,18050411810820992,88888,63050396551574951,39.758934,5.145366,6182.1760,3.726177,0.777221,-0.138147,0.746,-0.119236,2.596940,165.836717,-48.555807,8.992499,6.771897,5.916625,-1.922482
2,114877745,18262823713408000,87353,63050396551578779,40.065600,6.111945,6300.0860,4.088621,0.549031,-0.066147,1.063,-0.137236,25.483151,165.323549,-47.601931,49.278454,49.263408,-1.217621,19.025377
3,114843489,6142318629693312,88889,63050396551340305,40.354910,5.367575,6320.5684,3.780285,0.522622,-0.211147,0.863,-0.367236,2.156241,166.338417,-48.017155,25.287859,-9.792759,-23.314753,-16.186728
4,114842717,5947331409684480,89913,63050396551336895,40.378270,4.537912,6212.6494,4.224707,0.631931,-0.108147,1.133,-0.020236,2.388252,167.137705,-48.654791,12.340009,-4.436680,-11.514846,-1.950399
5,114843693,6180874550782720,88378,63050396551341048,40.477040,5.704103,6058.0693,4.196052,0.689712,-0.303147,0.388,-0.664236,3.465944,166.173808,-47.676827,9.962718,-2.427281,-9.662508,-2.125622
6,114843231,6077245580440704,89402,63050396551339120,40.582380,5.171726,6059.8193,4.038924,0.779513,-0.310147,0.190,-0.175236,2.844615,166.785601,-48.031643,40.820793,-12.879004,-38.735880,-23.094463
7,114841854,5738522984255488,90426,63050396551332952,40.859695,4.324028,6167.9740,3.829007,0.683892,0.098853,1.037,-0.074236,1.438452,167.905816,-48.520674,24.634876,23.954051,-5.751566,25.509668
8,114881192,19059312513209856,84794,63050396551593505,40.934300,7.823824,6026.6850,4.557385,0.825994,0.064853,0.646,0.299764,4.137614,164.826442,-45.715759,7.813338,-7.537335,-2.058355,7.810890
9,114842394,5864799318073728,89914,63050396551335398,40.952286,4.794470,6053.7803,4.243310,0.762052,-0.416147,0.290,-0.871236,2.826327,167.568659,-48.096944,8.717671,4.693461,-7.346375,-7.698774


Step 3: Calculate the distance of each star away from the center of the Milky Way

In [11]:
def distance_from_Earth(parallax_list):

    """
        This function returns a list of stellar distances from Earth in kpc, where the distances are the inverse of a specific star's
        parallax. Since parallax has units of arcseconds, the distance will initially have units in parsecs. In order to obtain units
        in kiloparsecs, we must divide the distance by 1000.

        Parameters:
            parallax_list (list) = List of parallax values (degrees)

        Returns:
            dist (list) = List of stellar distances from Earth (kiloparsec)
    """

    dist = []
    
    for p in parallax_list:
        d = (1 / p) / 1000

        dist.append(d)

    return dist

In [12]:
dist_vals = distance_from_Earth(df_cross_gaia['parallax'])

df_cross_gaia['dist_from_Earth(kpc)'] = dist_vals #Adds distance values to the dataframe

In [13]:
#Mask out distance values less than 0
good_dist = np.array(dist_vals) >= 0
df_cross_gaia = df_cross_gaia[good_dist]
df_cross_gaia = df_cross_gaia.reset_index(drop=True)

df_cross_gaia

,sdss_id,gaia_dr3_source_id,healpix,catalogid31,ra_1,dec_1,teff,logg,bp_rp,fe_h,n_h,o_h,parallax,l_2,b_2,pm,pmra_2,pmdec,dr2_radial_velocity,dist_from_Earth(kpc)
0,114876867,18047864894859136,89400,63050396551574904,39.364098,5.292768,5965.7430,3.904995,0.695686,-0.363451,0.398,-0.218161,1.514129,165.228239,-48.676291,10.255045,-8.348716,-5.955241,-21.060333,0.000660
1,114876879,18050411810820992,88888,63050396551574951,39.758934,5.145366,6182.1760,3.726177,0.777221,-0.138147,0.746,-0.119236,2.596940,165.836717,-48.555807,8.992499,6.771897,5.916625,-1.922482,0.000385
2,114877745,18262823713408000,87353,63050396551578779,40.065600,6.111945,6300.0860,4.088621,0.549031,-0.066147,1.063,-0.137236,25.483151,165.323549,-47.601931,49.278454,49.263408,-1.217621,19.025377,0.000039
3,114843489,6142318629693312,88889,63050396551340305,40.354910,5.367575,6320.5684,3.780285,0.522622,-0.211147,0.863,-0.367236,2.156241,166.338417,-48.017155,25.287859,-9.792759,-23.314753,-16.186728,0.000464
4,114842717,5947331409684480,89913,63050396551336895,40.378270,4.537912,6212.6494,4.224707,0.631931,-0.108147,1.133,-0.020236,2.388252,167.137705,-48.654791,12.340009,-4.436680,-11.514846,-1.950399,0.000419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,114818705,439942090065664,95552,63050396551230163,44.849308,1.364997,6115.7925,3.777392,0.744725,-0.231147,0.488,-0.290236,1.570016,175.334022,-48.052704,1.315554,-0.698750,-1.114643,-34.654488,0.000637
102,114817142,139363098967552,96576,63050396551224512,45.025726,1.110209,6076.8430,3.897285,0.728277,-0.322147,0.364,-0.256236,1.899869,175.790410,-48.107995,11.696977,-11.641076,-1.142190,6.287234,0.000526
103,114817041,121637769567616,96576,63050396551224191,45.238693,0.960671,6221.9950,3.892516,0.661037,-0.462147,0.663,-0.635236,2.353146,176.173672,-48.061622,5.362013,4.857900,2.269799,35.091732,0.000425
104,114817219,153935923508992,96577,63050396551224798,45.787930,0.828412,5912.9155,4.176083,0.812739,-0.116147,0.461,-0.122236,1.714762,176.886649,-47.760458,12.387972,-8.807361,-8.711615,-17.394642,0.000583


In [14]:
#Creating galactic coordinates in order to obtain stellar distance from the center of the Milky Way
dist_MW_vals = []

for i in range(len(df_cross_gaia['dist_from_Earth(kpc)'])):

    #original stellar coordinates
    ra = df_cross_gaia['ra_1'][i] * u.deg
    dec = df_cross_gaia['dec_1'][i] * u.deg
    dist = df_cross_gaia['dist_from_Earth(kpc)'][i] * u.kpc

    #Creating sky coordinates
    star_icrs = SkyCoord(ra = ra, dec = dec, distance = dist, frame = 'icrs')

    # Transform to the Galactocentric frame
    star_galactocentric = star_icrs.transform_to(Galactocentric())

    # Get the X, Y, Z coordinates in parsec
    X = star_galactocentric.cartesian.x
    Y = star_galactocentric.cartesian.y
    Z = star_galactocentric.cartesian.z

    #Find the radial distance of the star from the center of the Milky Way
    R = np.sqrt(np.square(X) + np.square(Y) + np.square(Z))

    #Add stellar radial distance to the list of distances to the center of the Milky Way
    dist_MW_vals.append(R.value)
    

In [15]:
df_cross_gaia['dist_from_center(kpc)'] = dist_MW_vals #Adds radial distance values to the dataframe

df_cross_gaia[:15]

,sdss_id,gaia_dr3_source_id,healpix,catalogid31,ra_1,dec_1,teff,logg,bp_rp,fe_h,...,o_h,parallax,l_2,b_2,pm,pmra_2,pmdec,dr2_radial_velocity,dist_from_Earth(kpc),dist_from_center(kpc)
0,114876867,18047864894859136,89400,63050396551574904,39.364098,5.292768,5965.7430,3.904995,0.695686,-0.363451,...,-0.218161,1.514129,165.228239,-48.676291,10.255045,-8.348716,-5.955241,-21.060333,0.000660,8.122422
1,114876879,18050411810820992,88888,63050396551574951,39.758934,5.145366,6182.1760,3.726177,0.777221,-0.138147,...,-0.119236,2.596940,165.836717,-48.555807,8.992499,6.771897,5.916625,-1.922482,0.000385,8.122247
2,114877745,18262823713408000,87353,63050396551578779,40.065600,6.111945,6300.0860,4.088621,0.549031,-0.066147,...,-0.137236,25.483151,165.323549,-47.601931,49.278454,49.263408,-1.217621,19.025377,0.000039,8.122026
3,114843489,6142318629693312,88889,63050396551340305,40.354910,5.367575,6320.5684,3.780285,0.522622,-0.211147,...,-0.367236,2.156241,166.338417,-48.017155,25.287859,-9.792759,-23.314753,-16.186728,0.000464,8.122301
4,114842717,5947331409684480,89913,63050396551336895,40.378270,4.537912,6212.6494,4.224707,0.631931,-0.108147,...,-0.020236,2.388252,167.137705,-48.654791,12.340009,-4.436680,-11.514846,-1.950399,0.000419,8.122270
5,114843693,6180874550782720,88378,63050396551341048,40.477040,5.704103,6058.0693,4.196052,0.689712,-0.303147,...,-0.664236,3.465944,166.173808,-47.676827,9.962718,-2.427281,-9.662508,-2.125622,0.000289,8.122189
6,114843231,6077245580440704,89402,63050396551339120,40.582380,5.171726,6059.8193,4.038924,0.779513,-0.310147,...,-0.175236,2.844615,166.785601,-48.031643,40.820793,-12.879004,-38.735880,-23.094463,0.000352,8.122229
7,114841854,5738522984255488,90426,63050396551332952,40.859695,4.324028,6167.9740,3.829007,0.683892,0.098853,...,-0.074236,1.438452,167.905816,-48.520674,24.634876,23.954051,-5.751566,25.509668,0.000695,8.122450
8,114881192,19059312513209856,84794,63050396551593505,40.934300,7.823824,6026.6850,4.557385,0.825994,0.064853,...,0.299764,4.137614,164.826442,-45.715759,7.813338,-7.537335,-2.058355,7.810890,0.000242,8.122163
9,114842394,5864799318073728,89914,63050396551335398,40.952286,4.794470,6053.7803,4.243310,0.762052,-0.416147,...,-0.871236,2.826327,167.568659,-48.096944,8.717671,4.693461,-7.346375,-7.698774,0.000354,8.122231


Step 5: Write the Plummer Potential equation and the circular velocity equations

In [16]:
def plummer_potential(r, b = 4.12, M = 1.29e12):

    """
        This function represents the Plummer Potential, which is a potential energy equation that is similar to Kepler's potential except
        that it's also dependent on the half-mass radius of a given galaxy. The Plummer Potential is used as a model for simple 
        Keplerian orbits within a spheroid. For this project, I am specifically modeling the orbits of stars within the Milky Way. Since
        the star's mass is negligibly small compared to the whole mass of the Milky Way, we don't need to include it. This is a simple
        model, because I am assuming the Milky Way's rotation is static and the stars won't interact/collide with other stars.

        Parameters:
            r (float) = Distance of a star from the Milky Way's center (kpc)
            b (float) = The Milky Way's half-mass radius (default for Milky Way = 4.12 kpc - based on Lian et. al 2024)
            M (float) = Mass of the Milky Way (most recent measurement = 1.29e12 solar masses - based on Grand et. al 2019)

        Returns:
            pot (float) = Potential energy of a star orbiting around the Milky Way (m^2/s^2 - energy per mass)
    
    """
    MW_mass = (M * u.solMass).to(u.kg) #Converts MW's mass from solar masses to kilograms, so units cancel in the potential equation
    
    pot = - (const.G * MW_mass) / np.sqrt(np.square((r * u.kpc).to(u.m)) + np.square((b * u.kpc).to(u.m)))

    return pot

In [17]:
#testing one star
test_pot = plummer_potential(df_cross_gaia['dist_from_center(kpc)'][0])

test_pot

<Quantity -6.09182597e+11 m2 / s2>

In [24]:
def deriv_plummer(r, b = 4.12, M = 1.29e12): #Apparently this function also represents the acceleration of the star in the radial direction

    """
        This function represents the radial derivative of the Plummer potential. This derivative will be plugged into the circular velocity
        equation, since the circular velocity is dependent on the star's distance from the Milky Way's center and the absolute value
        of the Plummer potential's radial derivative.

        Parameters:
            r (float) = Distance of a star from the Milky Way's center (kpc)
            b (float) = The Milky Way's half-mass radius (default for Milky Way = 4.12 kpc - based on Lian et. al 2024)
            M (float) = Mass of the Milky Way (most recent measurement = 1.29e12 solar masses - based on Grand et. al 2019)

        Returns:
            deriv_pot (float) = Time derivative of a star's potential energy while orbiting around the Milky Way (m^5/s^2)
        
    """

    MW_mass = (M * u.solMass).to(u.kg) #Converts MW's mass from solar masses to kilograms, so units cancel in the potential equation

    r = (r * u.kpc).to(u.m)
    b = (b * u.kpc).to(u.m)
    
    deriv_pot = - (const.G * MW_mass) * (r / ((np.square(r) + np.square(b)) ** (3/2)))

    return deriv_pot

In [25]:
test_deriv = deriv_plummer(df_cross_gaia['dist_from_center(kpc)'][0])

test_deriv

<Quantity -1.93319585e-09 m / s2>

In [28]:
def circular_velocity(r):

    """
        This function returns the circular velocity of a star orbiting the Milky Way, assuming that the Milky Way's rotation is static and
        the star doesn't interact/collide with other stars. This is a SIMPLE model; stars don't actually orbit around the galaxy in a
        perfect circle, since it has velocity components in more than just the radial direction. This equation will test whether my
        leapfrog method works before I model more elliptical orbits.

        Parameters:
            r (float) = Distance of a star from the Milky Way's center (kpc)

        Returns:
            v_circ (float) = Star's circular velocity measurement (m/s)
    """
    
    v_circ = np.sqrt((r * u.kpc).to(u.m) * np.abs(deriv_plummer(r)))

    return v_circ

In [29]:
test_circ = circular_velocity(df_cross_gaia['dist_from_center(kpc)'][0])

test_circ

<Quantity 696074.88844038 m / s>

Step 6: Implement the Leapfrog method (start with the circular velocity equation)

In [30]:
#Leapfrog method
def leapfrog_method(r, v, dt, b = 4.12, M = 1.29e12): #Debugged with Claude.ai
    """
        This function performs one integration step for the Leapfrog Method. I'm using the Leapfrog Method, since it conserves energy
        pretty well over long timescales. It's the preferred integrator for orbital simulations. 

        Parameters:
            r (float) = Current distance from the Milky Way's center (kpc)
            v (float) = Current velocity component (m/s)
            dt (float) = Time step (s)
            b (float) = The Milky Way's half-mass radius (default for Milky Way = 4.12 kpc - based on Lian et. al 2024)
            M (float) = Mass of the Milky Way (most recent measurement = 1.29e12 solar masses - based on Grand et. al 2019)

        Variables:
            a (float) = Current acceleration component (m/s^2)

        Returns:
            new_r (float) = New distance from the Milky Way's center (kpc - should be constant due to circular orbit)
            new_v (float) = New velocity component (m/s)
               
    """

    #Step 1: Half-step velocity update
    a = deriv_plummer(r)
    half_vel = v + (0.5 * a * dt) #changes units from m/s^2 to m/s

    #Step 2: Full-step position update
    new_r = r + (half_vel * dt)

    #Step 3: Calculate acceleration at new position
    new_a = deriv_plummer(new_r)

    #Step 4: Update the full velocity
    new_v = half_vel + (0.5 * new_a * dt)

    return new_r, new_v